## Introduction to global ocean biogeochemical modelling
<img src="boxmodel.png" width =800>

In [2]:
a = 6367e3;   # Earth radius           (m)
A = 4*pi*a^2; # Earth surface area     (m²)
d = 3700;     # ocean depth            (m)
V = 0.75*A*d; # volume of ocean        (m³)
h = 600;      # thickness of top layer (m)
#
dz = [h*ones(4,1);(d-h)*ones(4,1)]; # grid box thicknesses       (m)
dV = (dz/d).*((V/4)*ones(8,1));     # grid box volumes           (m³)
dAz = dV./dz                        # area of face ⟂ to z axis   (m²)
dy = sqrt.(dAz);                    # north-south side length    (m)
dx = sqrt.(dAz);                    # east-west side length      (m)
dAx = dV./dy;                       # area of face ⟂ to x axis   (m²)
dAy = dV./dx;                       # area of face ⟂ to y axis   (m²)
#
msk = [1, 1, 1, 0, 1, 1, 0, 0];     # wet-dry mask wet=1 dry = 0 
iwet = findall(x->x==1,msk);        # index to wet gridboxes
idry = findall(x->x==0,msk);        # index to dry gridboxes

In [13]:
using SparseArrays
TRdiv = spzeros(8,8);
# "Antarctic Circumpoloar Current"
acc = 100e6;  # (m³/s)
TRdiv += sparse([1,1],[1,3],dV[1]\[-acc,acc],8,8);
TRdiv += sparse([3,3],[3,1],dV[3]\[-acc,acc],8,8);
# "Meridional Overturning Circulation"
moc = 15e6    # (m³/s)
TRdiv += sparse([1,1],[1,5],dV[1]\[-moc,moc],8,8);
TRdiv += sparse([2,2],[2,1],dV[2]\[-moc,moc],8,8);
TRdiv += sparse([5,5],[5,6],dV[5]\[-moc,moc],8,8);
TRdiv += sparse([6,6],[6,2],dV[6]\[-moc,moc],8,8);
# vertical mixing at "high northern latitudes"
q = 10e6      # (m³/s)
TRdiv += sparse([2,2],[2,6],dV[2]\[-q,q],8,8);
TRdiv += sparse([6,6],[6,2],dV[6]\[-q,q],8,8);

In [14]:
Matrix(TRdiv)

8×8 Array{Float64,2}:
 -2.00662e-9    0.0           1.74489e-9  0.0  …   0.0          0.0  0.0
  2.61733e-10  -4.36222e-10   0.0         0.0      1.74489e-10  0.0  0.0
  1.74489e-9    0.0          -1.74489e-9  0.0      0.0          0.0  0.0
  0.0           0.0           0.0         0.0      0.0          0.0  0.0
  0.0           0.0           0.0         0.0      5.06581e-11  0.0  0.0
  0.0           8.44301e-11   0.0         0.0  …  -8.44301e-11  0.0  0.0
  0.0           0.0           0.0         0.0      0.0          0.0  0.0
  0.0           0.0           0.0         0.0      0.0          0.0  0.0

In [15]:
Matrix(TRdiv[iwet,iwet])

5×5 Array{Float64,2}:
 -2.00662e-9    0.0           1.74489e-9   2.61733e-10   0.0        
  2.61733e-10  -4.36222e-10   0.0          0.0           1.74489e-10
  1.74489e-9    0.0          -1.74489e-9   0.0           0.0        
  0.0           0.0           0.0         -5.06581e-11   5.06581e-11
  0.0           8.44301e-11   0.0          0.0          -8.44301e-11